In [1]:
import polars as pl

data = pl.read_parquet('tsa.parquet')

def clean_data():
    df = (data
    .with_columns(Year=pl.col('Date').dt.year(),
                Month=pl.col('Date').dt.month(),
                Day=pl.col('Date').dt.day())
    .with_columns(pl.when(pl.col('Month').eq(11) & pl.col('Day').eq(28))
                .then(pl.lit('Thanksgiving'))
                .when(pl.col('Month').eq(12) & pl.col('Day').eq(25))
                .then(pl.lit('Christmas'))
                .when(pl.col('Month').eq(7) & pl.col('Day').eq(4))
                .then(pl.lit('July 4th'))
                .when(pl.col('Month').eq(5) & pl.col('Day').eq(27))
                .then(pl.lit('Memorial Day'))
                .otherwise(None)
                .alias('Holiday')
                )
    )
    
    # Create high dataframe
    high_dfs = []
    for year in df['Year'].unique().to_list():
        high_df = pl.DataFrame({'Year':year,
                'Date':(df.filter(pl.col('Year') == year).filter(pl.col('Numbers') == pl.col('Numbers').max())['Date']),
                'Numbers':df.filter(pl.col('Year') == year)['Numbers'].max(),
                'Holiday':'Highest Record',
                })
        high_dfs.append(high_df)
    highest_df = pl.concat(high_dfs).with_columns(pl.col('Numbers').cast(pl.Int64))
    
    # Create low dataframe
    low_dfs = []
    for year in df['Year'].unique().to_list():
        low_df = pl.DataFrame({'Year':year,
                'Date':(df.filter(pl.col('Year') == year).filter(pl.col('Numbers') == pl.col('Numbers').min())['Date']),
                'Numbers':df.filter(pl.col('Year') == year)['Numbers'].min(),
                'Holiday':'Lowest Record',
                })
        low_dfs.append(low_df)
    lowest_df = pl.concat(low_dfs).with_columns(pl.col('Numbers').cast(pl.Int64))

    # Create total dataframe
    tot_dfs = []
    for year in df['Year'].unique().to_list():
        tot_df = pl.DataFrame({'Year':year,
                'Date':None,
                'Numbers':df.filter(pl.col('Year') == year)['Numbers'].sum(),
                'Holiday':'Annual Checkings',
                'Distribution':None
                })
        tot_dfs.append(tot_df)
    total_df = pl.concat(tot_dfs).with_columns(pl.col('Year').cast(pl.Int32))

    # Create holiday dataframe
    hol_dfs = []
    for year in df['Year'].unique().to_list():
        hol_df = (df
        .filter(pl.col('Year') == year)
        .filter(pl.col('Holiday').is_not_null())
        .select('Year','Date','Numbers','Holiday')
        )
        hol_dfs.append(hol_df)
    holiday_dfs = pl.concat(hol_dfs)

    # Create distribution dataframe
    dist_dfs = []
    for year in df['Year'].unique().to_list():
        dist_df = (df
        .filter(pl.col('Year') == year)
        .group_by('Month')
        .agg(pl.mean('Numbers'), pl.first('Year'))
        .sort('Month')
        .with_columns(Distribution=pl.col('Numbers').implode())
        .select('Year','Distribution').head(1)
        )
        dist_dfs.append(dist_df)
    distribution_df = pl.concat(dist_dfs)

    # Combine all dataframes and add Icon column.
    final_df = (pl.concat([holiday_dfs, highest_df, lowest_df])
    .join(distribution_df, on='Year', how='inner')
    .vstack(total_df)
    .with_columns(pl.when(pl.col('Holiday') == "Thanksgiving")
                .then(pl.col('Distribution'))
                .otherwise(None)
                .alias('Distribution')
                )
    .with_columns(pl.when(pl.col('Holiday') == "Memorial Day")
               .then(pl.lit('memorial.svg'))
               .when(pl.col('Holiday') == "July 4th")
               .then(pl.lit('flag.svg'))
               .when(pl.col('Holiday') == "Thanksgiving")
               .then(pl.lit('turkey.svg'))
               .when(pl.col('Holiday') == "Christmas")
               .then(pl.lit('christmas.svg'))
               .when(pl.col('Holiday') == "Highest Record")
               .then(pl.lit('high.svg'))
               .when(pl.col('Holiday') == "Lowest Record")
               .then(pl.lit('low.svg'))
               .otherwise(pl.lit('calendar.svg'))
               .alias('Icon')
               )
    .select('Icon', 'Holiday', 'Year', 'Date', 'Numbers', 'Distribution')
    .sort('Year')
    )
    return(final_df)

TSA = clean_data()
TSA

Icon,Holiday,Year,Date,Numbers,Distribution
str,str,i32,date,i64,list[f64]
"""memorial.svg""","""Memorial Day""",2019,2019-05-27,2512237,null
"""flag.svg""","""July 4th""",2019,2019-07-04,2088760,null
"""turkey.svg""","""Thanksgiving""",2019,2019-11-28,1591158,"[1.9902e6, 2.0906e6, … 2.3577e6]"
"""christmas.svg""","""Christmas""",2019,2019-12-25,1996541,null
"""high.svg""","""Highest Record""",2019,2019-12-01,2882915,null
…,…,…,…,…,…
"""turkey.svg""","""Thanksgiving""",2023,2023-11-28,2171943,"[1.9647e6, 2.0843e6, … 2.3407e6]"
"""christmas.svg""","""Christmas""",2023,2023-12-25,2028266,null
"""high.svg""","""Highest Record""",2023,2023-11-26,2908785,null


In [3]:
from great_tables import GT, html, loc, style, md, nanoplot_options

(
    GT(TSA, rowname_col="Icon", groupname_col="Year")
    .tab_stubhead(label=html('<b style="font-family: Inter, sans-serif; font-weight: 500;">Year</b>'))
    .tab_header(title=html('''
        <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;900&display=swap" rel="stylesheet">
        <h2 style="text-align:center; display: flex; align-items: center; justify-content: center; font-family: Inter, sans-serif; font-weight: 500; color: #014eac;">
            <img src="assets/plane3.svg" alt="Plane Icon" style="margin-right: 34px; height: 54px;">
            TSA Airport Checkings<br>on Major Holidays
            <img src="assets/plane3.svg" alt="Plane Icon" style="margin-left: 34px; height: 54px;">
        </h2>
    '''))
    .tab_options(container_width="100%",
                 table_background_color='#F0FFF0',
                 heading_background_color="#C0C0C0",
                 column_labels_background_color="#696969",
                 row_group_font_weight='bold',
                 row_group_background_color='#C0C0C0',
                 source_notes_font_size='12px',
                 row_group_padding='8px',
                 table_font_names='Inter')
    .cols_label(Date=html('<b style="font-family: Inter, sans-serif; font-weight: 500;">Date</b>'),
                Numbers=html('<b style="font-family: Inter, sans-serif; font-weight: 500;">Checkings</b>'),
                Distribution=html('<b style="font-family: Inter, sans-serif; font-weight: 500;">Avg Monthly Checkings</b>'),
                Holiday='')
    .fmt_number(columns='Numbers', decimals=0)
    .cols_width(cases={'Date':'120px'})
    .fmt_date(columns="Date", date_style="day_m")
    .tab_style(style=style.text(color='#556B2F', weight='bold'),
               locations=loc.body(rows=pl.col("Holiday") == "Annual Checkings"))
    .tab_style(style=style.text(color='black', weight='normal'),
               locations=loc.body(columns="Holiday"))
    .sub_missing(missing_text='')
    .fmt_nanoplot(columns="Distribution", reference_line="mean",
                  options=nanoplot_options(data_point_radius=12,
                                           data_point_stroke_color="black",
                                           data_point_stroke_width=4,
                                           data_point_fill_color="white",
                                           data_line_type="straight",
                                           data_line_stroke_color="brown",
                                           data_line_stroke_width=2,
                                           data_area_fill_color="#FF8C00",
                                           vertical_guide_stroke_color="green"))
    .fmt_image("Icon", path="assets")
    .tab_source_note(source_note=md("*Source:* [TSA Passenger Volumes](https://www.tsa.gov/travel/passenger-volumes)<br/>*Designer:* Joram Mutenge<br/>*www.jorammutenge.com*"))
)

GT(_tbl_data=shape: (35, 6)
┌───────────────┬──────────────────┬──────┬────────────┬───────────┬────────────────────────┐
│ Icon          ┆ Holiday          ┆ Year ┆ Date       ┆ Numbers   ┆ Distribution           │
│ ---           ┆ ---              ┆ ---  ┆ ---        ┆ ---       ┆ ---                    │
│ str           ┆ str              ┆ i32  ┆ date       ┆ i64       ┆ list[f64]              │
╞═══════════════╪══════════════════╪══════╪════════════╪═══════════╪════════════════════════╡
│ memorial.svg  ┆ Memorial Day     ┆ 2019 ┆ 2019-05-27 ┆ 2512237   ┆ null                   │
│ flag.svg      ┆ July 4th         ┆ 2019 ┆ 2019-07-04 ┆ 2088760   ┆ null                   │
│ turkey.svg    ┆ Thanksgiving     ┆ 2019 ┆ 2019-11-28 ┆ 1591158   ┆ [1.9902e6, 2.0906e6, … │
│               ┆                  ┆      ┆            ┆           ┆ 2.3577e…               │
│ christmas.svg ┆ Christmas        ┆ 2019 ┆ 2019-12-25 ┆ 1996541   ┆ null                   │
│ high.svg      ┆ Highest Record   ┆ 2019 ┆ 2019-12-01 ┆ 2882915   ┆ null                   │
│ …             ┆ …                ┆ …    ┆ …          ┆ …         ┆ …                      │
│ turkey.svg    ┆ Thanksgiving     ┆ 2023 ┆ 2023-11-28 ┆ 2171943   ┆ [1.9647e6, 2.0843e6, … │
│               ┆                  ┆      ┆            ┆           ┆ 2.3407e…               │
│ christmas.svg ┆ Christmas        ┆ 2023 ┆ 2023-12-25 ┆ 2028266   ┆ null                   │
│ high.svg      ┆ Highest Record   ┆ 2023 ┆ 2023-11-26 ┆ 2908785   ┆ null                   │
│ low.svg       ┆ Lowest Record    ┆ 2023 ┆ 2023-01-31 ┆ 1534786   ┆ null                   │
│ calendar.svg  ┆ Annual Checkings ┆ 2023 ┆ null       ┆ 858548196 ┆ null                   │
└───────────────┴──────────────────┴──────┴────────────┴───────────┴────────────────────────┘, _body=<great_tables._gt_data.Body object at 0x000001DF64D5E420>, _boxhead=Boxhead([ColInfo(var='Icon', type=<ColInfoTypeEnum.stub: 2>, column_label='Icon', column_align='left', column_width=None), ColInfo(var='Holiday', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None), ColInfo(var='Year', type=<ColInfoTypeEnum.row_group: 3>, column_label='Year', column_align='right', column_width=None), ColInfo(var='Date', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='<b style="font-family: Inter, sans-serif; font-weight: 500;">Date</b>', type='html'), column_align='right', column_width='120px'), ColInfo(var='Numbers', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='<b style="font-family: Inter, sans-serif; font-weight: 500;">Checkings</b>', type='html'), column_align='right', column_width=None), ColInfo(var='Distribution', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='<b style="font-family: Inter, sans-serif; font-weight: 500;">Avg Monthly Checkings</b>', type='html'), column_align='center', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=2019, rowname='memorial.svg', group_label=None, built=False), RowInfo(rownum_i=1, group_id=2019, rowname='flag.svg', group_label=None, built=False), RowInfo(rownum_i=2, group_id=2019, rowname='turkey.svg', group_label=None, built=False), RowInfo(rownum_i=3, group_id=2019, rowname='christmas.svg', group_label=None, built=False), RowInfo(rownum_i=4, group_id=2019, rowname='high.svg', group_label=None, built=False), RowInfo(rownum_i=5, group_id=2019, rowname='low.svg', group_label=None, built=False), RowInfo(rownum_i=6, group_id=2019, rowname='calendar.svg', group_label=None, built=False), RowInfo(rownum_i=7, group_id=2020, rowname='memorial.svg', group_label=None, built=False), RowInfo(rownum_i=8, group_id=2020, rowname='flag.svg', group_label=None, built=False), RowInfo(rownum_i=9, group_id=2020, rowname='turkey.svg', group_label=None, built=False), RowInfo(rownum_i=10, group_id=2020, rowname='christmas.svg', group_label=None, built=False), RowInfo(rownum_i=11, group_id=2020, rowname='high.svg', group_label=None, built=False), RowInfo(r